# 数据的合并与重塑

## 1.concat 
concat 函数是在pandas下的方法，可以将数据根据不同的数据轴进行简单的合并。其函数原型如下：

pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
       keys=None, levels=None, names=None, verify_integrity=False)
       
相关参数说明：

objs: series，dataframe或者是panel构成的序列lsit 

axis： 需要合并链接的轴，0是行，1是列 

join：连接的方式 inner，或者outer

### 1.1 相同字段的表首尾相接
如下：

In [56]:
# 导入相关模块
import numpy as np
import pandas as pd

In [57]:
ratings_1 = pd.read_csv("../data/ml-latest-small/ratings_1.csv")
ratings_2 = pd.read_csv("../data/ml-latest-small/ratings_2.csv")
df1 = ratings_1.head(5)
df2 = ratings_2.tail(5)
# 将df1 和df2 组成
farme = [df1, df2]
# concat 参数需要是 list 
result = pd.concat(farme)
result

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
49997,671,6268,2.5,1065579370
49998,671,6269,4.0,1065149201
49999,671,6365,4.0,1070940363
50000,671,6385,2.5,1070979663
50001,671,6565,3.5,1074784724


In [58]:
# 要在相接的时候在加上一个层次的key来识别数据源自于哪张表，可以增加key参数
result_1 = pd.concat(farme, keys=['x', 'y'])
result_1

userId  movieId  rating   timestamp
x 0           1       31     2.5  1260759144
  1           1     1029     3.0  1260759179
  2           1     1061     3.0  1260759182
  3           1     1129     2.0  1260759185
  4           1     1172     4.0  1260759205
y 49997     671     6268     2.5  1065579370
  49998     671     6269     4.0  1065149201
  49999     671     6365     4.0  1070940363
  50000     671     6385     2.5  1070979663
  50001     671     6565     3.5  1074784724

### 1.2 横向表拼接--- 行对齐
#### 1.2.1 axis
当axis = 1的时候，concat就是行对齐，然后将不同列名称的两张表合并

其中，会根据表格内容进行合并，无数据区域以 np.nan 填充.

In [59]:
movies = pd.read_csv("../data/ml-latest-small/movies.csv")
df3 = movies.head(10)
result_2 = pd.concat([df1, df3], axis=1)
result_2

,userId,movieId,rating,timestamp,movieId,title,genres
0,1.0,31.0,2.5,1.260759e+09,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1.0,1029.0,3.0,1.260759e+09,2,Jumanji (1995),Adventure|Children|Fantasy
2,1.0,1061.0,3.0,1.260759e+09,3,Grumpier Old Men (1995),Comedy|Romance
3,1.0,1129.0,2.0,1.260759e+09,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1.0,1172.0,4.0,1.260759e+09,5,Father of the Bride Part II (1995),Comedy
5,NaN,NaN,NaN,NaN,6,Heat (1995),Action|Crime|Thriller
6,NaN,NaN,NaN,NaN,7,Sabrina (1995),Comedy|Romance
7,NaN,NaN,NaN,NaN,8,Tom and Huck (1995),Adventure|Children
8,NaN,NaN,NaN,NaN,9,Sudden Death (1995),Action
9,NaN,NaN,NaN,NaN,10,GoldenEye (1995),Action|Adventure|Thriller


#### 1.2.2 join
加上join参数的属性，如果为’inner’得到的是两表的交集，如果是outer，得到的是两表的并集。

其中，会根据表格内容进行合并，无数据区域以 np.nan 填充.

In [60]:
result_3 = pd.concat([df1, df3], axis=1, join='inner')
result_3

,userId,movieId,rating,timestamp,movieId,title,genres
0,1,31,2.5,1260759144,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1029,3.0,1260759179,2,Jumanji (1995),Adventure|Children|Fantasy
2,1,1061,3.0,1260759182,3,Grumpier Old Men (1995),Comedy|Romance
3,1,1129,2.0,1260759185,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,1172,4.0,1260759205,5,Father of the Bride Part II (1995),Comedy


#### 1.2.3 join_axes
如果有join_axes的参数传入，可以指定根据那个轴来对齐数据 

例如根据df1表对齐数据，就会保留指定的df1表的轴，然后将df4的表与之拼接

In [61]:
links = pd.read_csv("../data/ml-latest-small/links.csv")
df4 = links.head(8)
result_4 = pd.concat([df1, df4], axis=1, join_axes=[df1.index])
result_4

,userId,movieId,rating,timestamp,movieId,imdbId,tmdbId
0,1,31,2.5,1260759144,1,114709.0,862.0
1,1,1029,3.0,1260759179,2,113497.0,8844.0
2,1,1061,3.0,1260759182,3,113228.0,NaN
3,1,1129,2.0,1260759185,4,114885.0,31357.0
4,1,1172,4.0,1260759205,5,113041.0,11862.0


### 1.3 append
append是series和dataframe的方法，使用它就是默认沿着列进行凭借（axis = 0，列对齐）

In [62]:
result_5 = df1.append(df2)
result_5

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
49997,671,6268,2.5,1065579370
49998,671,6269,4.0,1065149201
49999,671,6365,4.0,1070940363
50000,671,6385,2.5,1070979663
50001,671,6565,3.5,1074784724


### 1.4 无视index的concat
如果两个表的index都没有实际含义，使用ignore_index参数，置true，合并的两个表就睡根据列字段对齐，然后合并。最后再重新整理一个新的index。

In [63]:
result_6 = pd.concat([df1, df2], ignore_index=True)
result_6

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,671,6268,2.5,1065579370
6,671,6269,4.0,1065149201
7,671,6365,4.0,1070940363
8,671,6385,2.5,1070979663
9,671,6565,3.5,1074784724


### 1.5 合并的同时增加区分数据组的键
前面提到的keys参数可以用来给合并后的表增加key来区分不同的表数据来源
#### 1.5.1 可以直接用key参数实现

In [64]:
result_7 = pd.concat([df1, df2], keys=['x', 'y'])
result_7

userId  movieId  rating   timestamp
x 0           1       31     2.5  1260759144
  1           1     1029     3.0  1260759179
  2           1     1061     3.0  1260759182
  3           1     1129     2.0  1260759185
  4           1     1172     4.0  1260759205
y 49997     671     6268     2.5  1065579370
  49998     671     6269     4.0  1065149201
  49999     671     6365     4.0  1070940363
  50000     671     6385     2.5  1070979663
  50001     671     6565     3.5  1074784724

#### 1.5.2 传入字典来增加分组键

In [65]:
pieces = {'x': df1, 'y': df2}
result_8 = pd.concat(pieces)
result_8

userId  movieId  rating   timestamp
x 0           1       31     2.5  1260759144
  1           1     1029     3.0  1260759179
  2           1     1061     3.0  1260759182
  3           1     1129     2.0  1260759185
  4           1     1172     4.0  1260759205
y 49997     671     6268     2.5  1065579370
  49998     671     6269     4.0  1065149201
  49999     671     6365     4.0  1070940363
  50000     671     6385     2.5  1070979663
  50001     671     6565     3.5  1074784724

### 1.6 在dataframe中加入新的行
append方法可以将 series 和 字典就够的数据作为dataframe的新一行插入。

In [66]:
s = pd.Series([2, 40, 3.0, 1234567], index=['userId', 'movieId', 'rating', 'timestamp'])
result_9 = df1.append(s, ignore_index=True)
result_9

,userId,movieId,rating,timestamp
0,1.0,31.0,2.5,1.260759e+09
1,1.0,1029.0,3.0,1.260759e+09
2,1.0,1061.0,3.0,1.260759e+09
3,1.0,1129.0,2.0,1.260759e+09
4,1.0,1172.0,4.0,1.260759e+09
5,2.0,40.0,3.0,1.234567e+06


## 2.merge
pandas的merge方法提供了一种类似于SQL的内存链接操作，官网文档提到它的性能会比其他开源语言的数据操作（例如R）要高效。其函数原型如下：

merge(left,right,how='inner',on=None,left_on=None,right_on=None,
    left_index=False,right_index=False,sort=False,suffixes=('_x','_y'),copy=True)

参数说明：

on：列名，join用来对齐的那一列的名字，用到这个参数的时候一定要保证左表和右表用来对齐的那一列都有相同的列名。

left_on：左表对齐的列，可以是列名，也可以是和dataframe同样长度的arrays。

right_on：右表对齐的列，可以是列名，也可以是和dataframe同样长度的arrays。

left_index/ right_index: 如果是True的haunted以index作为对齐的key

how：数据融合的方法。

sort：根据dataframe合并的keys按字典顺序排序，默认是，如果置false可以提高表现。

merge的默认合并方法：
    merge用于表内部基于 index-on-index 和 index-on-column(s) 的合并，但默认是基于index来合并。

In [67]:
result_10 = pd.merge(df3, df4, on=['movieId'])
result_10

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709.0,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497.0,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228.0,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885.0,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041.0,11862.0
5,6,Heat (1995),Action|Crime|Thriller,113277.0,949.0
6,7,Sabrina (1995),Comedy|Romance,NaN,11860.0
7,8,Tom and Huck (1995),Adventure|Children,112302.0,45325.0


没有指定how的话默认使用inner方法。

how的方法有：left, right, outer, inner。

### 2.1 how 方法，选择合并方式
#### 2.1.1 left
只保留左表的所有数据

In [68]:
result_11 = pd.merge(df3, df4, how='left', on=['movieId'])
result_11

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709.0,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497.0,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228.0,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885.0,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041.0,11862.0
5,6,Heat (1995),Action|Crime|Thriller,113277.0,949.0
6,7,Sabrina (1995),Comedy|Romance,NaN,11860.0
7,8,Tom and Huck (1995),Adventure|Children,112302.0,45325.0
8,9,Sudden Death (1995),Action,NaN,NaN
9,10,GoldenEye (1995),Action|Adventure|Thriller,NaN,NaN


#### 2.1.2 right
只保留右表的所有数据

In [69]:
result_12 = pd.merge(df3, df4, how='right', on=['movieId'])
result_12

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709.0,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497.0,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228.0,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885.0,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041.0,11862.0
5,6,Heat (1995),Action|Crime|Thriller,113277.0,949.0
6,7,Sabrina (1995),Comedy|Romance,NaN,11860.0
7,8,Tom and Huck (1995),Adventure|Children,112302.0,45325.0


#### 2.1.3 outer
保留两个表的所有信息

In [70]:
result_13 = pd.merge(df3, df4, how='outer', on=['movieId'])
result_13

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709.0,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497.0,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228.0,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885.0,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041.0,11862.0
5,6,Heat (1995),Action|Crime|Thriller,113277.0,949.0
6,7,Sabrina (1995),Comedy|Romance,NaN,11860.0
7,8,Tom and Huck (1995),Adventure|Children,112302.0,45325.0
8,9,Sudden Death (1995),Action,NaN,NaN
9,10,GoldenEye (1995),Action|Adventure|Thriller,NaN,NaN


#### 2.1.4 inner
只保留两个表中公共部分的信息

In [71]:
result_14 = pd.merge(df3, df4, how='inner', on=['movieId'])
result_14

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709.0,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497.0,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228.0,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885.0,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041.0,11862.0
5,6,Heat (1995),Action|Crime|Thriller,113277.0,949.0
6,7,Sabrina (1995),Comedy|Romance,NaN,11860.0
7,8,Tom and Huck (1995),Adventure|Children,112302.0,45325.0


### 2.2 indicator
v0.17.0 版本的pandas开始还支持一个indicator的参数，如果置True的时候，输出结果会增加一列 ’ _merge’。_merge列可以取三个值

left_only 只在左表中

right_only 只在右表中

both 两个表中都有

### 2.3 join方法
dataframe内置的join方法是一种快速合并的方法。它默认以index作为对齐的列。

#### 2.3.1 how 参数
join中的how参数和merge中的how参数一样，用来指定表合并保留数据的规则。

具体可见前面的 how 说明。

#### 2.3.2 on 参数
在实际应用中如果右表的索引值正是左表的某一列的值，这时可以通过将 右表的索引 和 左表的列 对齐合并这样灵活的方式进行合并。

In [72]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'key': ['K0', 'K1', 'K0', 'K1']})
right = pd.DataFrame({'C': ['C0', 'C1'],
                    'D': ['D0', 'D1']},
                    index=['K0', 'K1'])
result_15 = left.join(right, on='key')
result_15

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K0,C0,D0
3,A3,B3,K1,C1,D1


#### 2.3.3 suffix后缀参数
如果和表合并的过程中遇到有一列两个表都同名，但是值不同，合并的时候又都想保留下来，就可以用suffixes给每个表的重复列名增加后缀。

In [73]:
left = pd.DataFrame({'k':['k0', 'k1', 'k2'],
                    'v':[0, 1, 2]})
right = pd.DataFrame({'k':['k0', 'k0', 'k3'],
                    'v':[3, 4, 5]})
result_16 = pd.merge(left, right, on='k', suffixes=['_l', '_r'])
result_16

,k,v_l,v_r
0,k0,0,3
1,k0,0,4


### 2.4 组合多个dataframe
一次组合多个dataframe的时候可以传入元素为dataframe的列表或者tuple。一次join多个，一次解决多次烦恼~

In [74]:
left = pd.DataFrame({'v': [1, 2, 3]}, index=['k0', 'k1', 'k2'])
right = pd.DataFrame({'v': [4, 5, 6]}, index=['k0', 'k0', 'k3'])
right2 = pd.DataFrame({'v': [7, 8, 9]}, index=['k1', 'k1', 'k2'])
right2
result_17 = left.join([right, right2])
result_17

,v_x,v_y,v
k0,1,4.0,NaN
k0,1,5.0,NaN
k1,2,NaN,7.0
k1,2,NaN,8.0
k2,3,NaN,9.0


### 1.5 更新表的nan值
#### 1.5.1 combine_first
如果一个表的nan值，在另一个表相同位置（相同索引和相同列）可以找到，则可以通过combine_first来更新数据

#### 1.5.2 update
如果要用一张表中的数据来更新另一张表的数据则可以用update来实现

#### 1.5.3 combine_first 和 update 的区别
使用combine_first会只更新左表的nan值。而update则会更新左表的所有能在右表中找到的值（两表位置相对应）。